In [1]:
from datasets import load_dataset, load_metric
import random
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")


In [3]:
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
questions = [
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]
for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    pred = model(**inputs)
    print(pred)
    answer_start_scores, answer_end_scores = pred['start_logits'][0] ,pred['end_logits'][0]

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-6.6028, -5.8443, -7.8482, -7.8212, -8.6835, -8.8632, -8.6049, -8.1944,
         -8.1338, -7.6590, -7.8377, -9.2932, -6.6028, -4.0176, -2.9258, -7.7681,
         -6.9620, -7.9091, -8.1943, -5.9419, -7.8839, -8.2694, -7.8974, -8.0646,
         -5.1529, -8.1331, -6.3936, -8.0166, -8.2358, -8.0699, -8.1815, -6.2712,
         -7.9358, -8.1978, -8.1440, -5.6098, -6.9490, -6.5179, -6.2459, -7.6979,
         -7.7173, -6.1049, -7.6189, -7.2503, -5.4475, -8.1642, -6.3496, -8.0311,
         -8.0922, -7.0914, -8.2821, -6.7683, -8.4299, -6.6811, -8.0168, -8.4784,
         -7.0992, -8.0497, -8.0467, -8.4790, -7.0054, -7.7931, -6.3428, -6.6257,
         -7.1404, -5.9187, -6.9042, -7.0796, -8.0231, -6.6892, -8.0582, -7.7207,
         -8.2388, -5.4746, -6.7801, -6.1311, -7.8433, -6.1570, -7.3047, -5.2370,
         -2.5079,  5.3793,  5.0301, -3.1776, -4.6009, -6.1616, -6.0828, -3.6223,
         -5.2340, -2.4337, -7.2006, -5.7583, -7.5441, -5

In [2]:
datasets = load_dataset("squad")
def visualize(datasets, datatype = 'train', n_questions=10):
    n = len(datasets[datatype])
    random_questions=random.choices(list(range(n)),k=n_questions)
    for i in random_questions:
        print(f"Context:{datasets[datatype][i]['context']}")
        print(f"Question:{datasets[datatype][i]['question']}")
        print(f"Answer:{datasets[datatype][i]['answers']['text']}")
        print(f"Answer Start in Text:{datasets[datatype][i]['answers']['answer_start']}")
        print("-"*100)
visualize(datasets)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /home/abdelrahman/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Context:On 18 January 2014, the interim government successfully institutionalised a more secular constitution. The president is elected to a four-year term and may serve 2 terms. The parliament may impeach the president. Under the constitution, there is a guarantee of gender equality and absolute freedom of thought. The military retains the ability to appoint the national Minister of Defence for the next 8 years. Under the constitution, political parties may not be based on "religion, race, gender or geography".
Question:Who may impeach the president?
Answer:['parliament']
Answer Start in Text:[175]
----------------------------------------------------------------------------------------------------
Context:The Russians avoided Napoleon's objective of a decisive engagement and instead retreated deeper into Russia. A brief attempt at resistance was made at Smolensk in August; the Russians were defeated in a series of battles, and Napoleon resumed his advance. The Russians again avoided b

In [25]:
datasets['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [8]:
datasets['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [21]:
datasets['train'][1100]

{'id': '57341b484776f41900661888',
 'title': 'Montana',
 'context': 'Montana is home to a diverse array of fauna that includes 14 amphibian, 90 fish, 117 mammal, 20 reptile and 427 bird species. Additionally, there are over 10,000 invertebrate species, including 180 mollusks and 30 crustaceans. Montana has the largest grizzly bear population in the lower 48 states. Montana hosts five federally endangered species–black-footed ferret, whooping crane, least tern, pallid sturgeon and white sturgeon and seven threatened species including the grizzly bear, Canadian lynx and bull trout. The Montana Department of Fish, Wildlife and Parks manages fishing and hunting seasons for at least 17 species of game fish including seven species of trout, walleye and smallmouth bass and at least 29 species of game birds and animals including ring-neck pheasant, grey partridge, elk, pronghorn antelope, mule deer, whitetail deer, gray wolf and bighorn sheep.',
 'question': 'How many species of game fish have

In [15]:
type(datasets['train'])

datasets.arrow_dataset.Dataset

In [ ]:
datasets = load_dataset("squad")
def visualize(datasets, datatype = 'train', n_questions=10):
    n = len(datasets[datatype])
    random_questions=random.choices(list(range(n)),k=n_questions)
    for i in random_questions:
        print(f"Context:{datasets[datatype][i]['context']}")
        print(f"Question:{datasets[datatype][i]['question']}")
        print(f"Answer:{datasets[datatype][i]['answers']['text']}")
        print(f"Answer Start in Text:{datasets[datatype][i]['answers']['answer_start']}")
        print("-"*100)
visualize(datasets)